### 3 차 전처리_원택님
* 1. 분반번호 쉼표 나열
* 2. 학년과 분반 분리
    a. 학년 칼럼 추가 
* 3. 외국인재학생 -> T/F
* 4. 내국인재학생 -> T/F

In [ ]:
import pandas as pd
import re
# 다양한 인코딩을 시도해보기
encodings = ['euc-kr']

for enc in encodings:
    try:
        df = pd.read_csv('df_result.csv', encoding=enc)
        print(f"Successfully read with encoding: {enc}")
        print(df.head())
        break
    except UnicodeDecodeError:
        print(f"Failed to read with encoding: {enc}")

In [ ]:
# 새로운 열 추가를 위한 함수 정의
def extract_grade_info(row):
    grades = []
    if '1학년' in row:
        grades.append('1')
    if '2학년' in row:
        grades.append('2')
    if '3학년' in row:
        grades.append('3')
    if '4학년' in row:
        grades.append('4')
    return ','.join(grades)

# 새로운 열 '학년 정보' 추가
df['학년'] = df['분반'].apply(extract_grade_info)
df['학년'].value_counts()

In [ ]:
# # 패턴을 인식하고 대체하는 함수 정의
# def expand_ranges(text):
#     def replace_range(match):
#         start_char = match.group(1)
#         start_num = int(match.group(2))
#         end_num = int(match.group(3))
#         expanded = ','.join(f"{start_char}{i}" for i in range(start_num, end_num + 1))
#         return expanded
    
#     # 정규 표현식을 사용하여 패턴을 인식하고 대체
#     pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')
#     result = pattern.sub(replace_range, text)
    
#     return result

def expand_ranges(text):
    # 정규 표현식을 사용하여 패턴을 인식하고 대체
    pattern = re.compile(r'([A-Z])(\d+)~\1(\d+)')

    # 각 패턴에 대해 확장된 범위로 대체하고, 그 결과만 반환
    def replace_range(match):
        start_char = match.group(1)
        start_num = int(match.group(2))
        end_num = int(match.group(3))
        return ','.join(f"{start_char}{i}" for i in range(start_num, end_num + 1))
    
    # 전체 문자열 대신, 일치하는 부분만 찾아서 리스트로 모아 반환
    expanded_parts = pattern.findall(text)
    expanded_results = [replace_range(match) for match in pattern.finditer(text)]
    
    return ','.join(expanded_results)  # 리스트를 문자열로 결합
def replace_spaces_with_commas(text):
    # 모든 공백을 콤마로 대체
    return text.replace(" ", ",")

# '분반' 열에 함수 적용
df['분반_학과'] = df['분반'].apply(expand_ranges)

# 결과 출력
print(df)

In [ ]:
# 조건에 따라 열 값 수정
conditions_foreign = ['외국인만 수강가능', '공대 합반,외국인만수강가능', '전정대와 합반 외국인만 수강가능']
condition_domestic = '외국인학생 수강불가'


# 외국인재학생 = True, 국내재학생 = False 설정
for condition in conditions_foreign:
    df.loc[df['분반'] == condition, ['외국인재학생', '국내재학생']] = [True, False]


# 외국인재학생 = False, 국내재학생 = True 설정
df.loc[df['분반'] == condition_domestic, ['외국인재학생', '국내재학생']] = [False, True]

In [ ]:
# # '외국인전용' 열 추가
# df['외국인전용'] = df['분반'].apply(lambda x: True if any(term in x for term in ['외국인만 수강가능', '공대 합반,외국인만수강가능', '전정대와 합반 외국인만 수강가능']) else False)

# # '국내재학생' 열 추가
# df['국내재학생'] = ~df['외국인전용']

In [ ]:
# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['참빛인재대학'] = df['분반'].apply(lambda x: 'TRUE' if '참빛인재대학' in x else pd.NA)


# '참빛인재대학'이 포함되어 있는지 확인하고, 조건에 따라 새 열에 'TRUE' 할당
df['체육특기자'] = df['분반'].apply(lambda x: 'TRUE' if '체육특기자' in x else pd.NA)

In [ ]:
# 삭제할 문자열 리스트
remove_items = ['체육실기', '음악실기', '미술실기']

# 특정 문자열 삭제
for item in remove_items:
    df['분반'] = df['분반'].str.replace(item, '')

In [ ]:
# 결과를 CSV 파일로 저장
df.to_csv('df_result_expanded1.csv', index=False, encoding='cp949')

In [ ]:
# '외국인'이 포함된 행 필터링
foreign_students = df[df['분반'].str.contains('체육특기')]

# '외국인'이 포함된 유니크한 값 추출
unique_foreign_classes = foreign_students['분반'].unique()

# 결과 출력
print("분반에 포함된 유니크한 값:")
print(unique_foreign_classes)

In [ ]:
재수강용 

외국인 분반에 포함된 유니크한 값:
['참빛인재대학 전용' '참빛인재대학']

['체육실기' '음악실기' '미술실기']

['체육특기자만 수강가능']